In [1]:
import netCDF4 as nc
import glob as glob
import pylab as py
import numpy as np
import hickle as hkl
import cartopy.crs as ccrs
import cartopy.util as cutil
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
import matplotlib
import sys
import logging
import time as ti
import scipy.signal as signal
from scipy import stats
from scipy.linalg import lstsq
import matplotlib as mpl
import numbers
import importlib
from tqdm import tqdm
import seaborn as sns

sys.path.append('/data/pbarpanda/python_scripts/modules/')

import momentum_advection_class as momentum_advect
import logruns as logruns
import save_and_load_hdf5_files as h5saveload
import eulerian_fluxes as eflux
# sys.path.append('/data/pbarpanda/python_scripts/rerun_ERAI_data_extraction/Aug11_on_Georges_data_calculate_mean_fields_for_different_MJO_phases/')
# import Aug11_netcdf_utilities_Georges_data as ncutil
import Aug11_netcdf_utilities_TRACK as ncutil
import temporal_filter_for_all_years_in_one_level_with_seasonal_cycle as tf

cmap = matplotlib.colors.LinearSegmentedColormap.from_list("", 
      [  "darkred", "darkorange", "pink", "white", "white","skyblue", "dodgerblue", "navy"][::-1])

def colorbar(fontsize=20):
    cbar = py.colorbar()
    for t in cbar.ax.get_yticklabels():
         t.set_fontsize(fontsize)
            
import os
os.environ["HDF5_USE_FILE_LOCKING"] = 'FALSE'

three_month_combos = { 'DJF':(12,1,2), 'MAM':(3,4,5),  'JJA':(6,7,8),  'SON':(9,10,11), \
                       'JFM':(1,2,3),  'AMJ':(4,5,6),  'JAS':(7,8,9),  'OND':(10,11,12), \
                       'FMA':(2,3,4),  'MJJ':(5,6,7),  'ASO':(8,9,10), 'NDJ':(11,12,1)}

month_string       = ['Jan ', 'Feb ', 'Mar ', 'Apr ', 'May ', 'Jun ', 'Jul ', 'Aug ', 'Sep ', 'Oct ', 'Nov ', 'Dec ']

lat_lon = h5saveload.load_dict_from_hdf5('./lat_lon.hdf5')
lon, lat = lat_lon['lon'], lat_lon['lat']

### Load binned data

In [2]:
dirname = '../raw_omi_and_raw_U_many_bins/'
filenames = ['monthly_regression_u_DJF', 'monthly_Y_u_DJF', 'monthly_Y_u_DJF_seas', 'monthly_Y_u_DJF_not_mean', 'monthly_X_DJF']
for filename in tqdm(filenames):
    filename2 = dirname+filename
    h5saveload.make_sure_path_exists(os.path.dirname(filename2))
    vars()[filename] = h5saveload.load_dict_from_hdf5(filename2)

100%|██████████| 5/5 [00:02<00:00,  1.74it/s]


### A function to plot pdf/histogram of jet location

In [3]:
def make_map_and_box(jet_location = [-90, 90, 100, 250], figsize=(20,8), xlims = ['0.0', '-2.5'], ylims = [-20, 60]):
    
    
        import matplotlib.patches as mpatches
        ymin, ymax, xmin, xmax = jet_location
        left, bottom, width, height = (xmin, ymin, xmax-xmin, ymax-ymin)

    
        E_W_lon      = (lon + 180) % 360 - 180
        EW_lon_dict  = {}
        
        for i in range(len(lon)):
            EW_lon_dict[ str(E_W_lon[i]) ] = lon[i]

        py.rcParams.update({
                    "text.usetex": True,
                    "font.family": "serif",
                    "font.serif": ["Palatino"],
                })


        CL = 200
        projection0  = ccrs.PlateCarree
        projection   = ccrs.PlateCarree(central_longitude=200, )

        fig, axs = py.subplots(1, 1, figsize=figsize, \
                               sharey=True, sharex=True, \
                               constrained_layout=True, \
                               subplot_kw={'projection': projection})


        A=-1
        for AX in [axs]:
            A = A+1

            mean = 0; s=0
            for key in monthly_Y_u_DJF_seas['ensoall']['phase_all'].keys():
                mean  =  mean + (monthly_Y_u_DJF_seas['ensoall']['phase_all'][key].mean(axis=0)); s=s+1
            mean = mean/s  

            cyclic_data, cyclic_lons = cutil.add_cyclic_point(data=(mean), coord=lon)
            c=AX.contour( cyclic_lons, lat, cyclic_data,  [10, 20, 30, 40, 50, 60, 70], cmap=py.cm.binary, linewidths=2, transform=projection0()); 
            
            rect=mpatches.Rectangle((left,bottom),width,height, 
                                    fill=False,
                                    color="purple",
                                    linewidth=4, transform=ccrs.PlateCarree())
            AX.add_patch(rect)            
                    
            AX.clabel(c, fmt='%1d', fontsize=15)
            AX.coastlines(color='brown', alpha=1)
            AX.set_aspect('auto')
            xlims =xlims; ylims = ylims
            minlon = EW_lon_dict[xlims[0]]  #1 #-120 + 180 #CL
            maxlon = EW_lon_dict[xlims[1]]  #359 # +60 + 180 #CL
            AX.set_extent([minlon, maxlon, ylims[0], ylims[-1]], ccrs.PlateCarree())

            AX.tick_params(labelsize=15)

#             no_of_data_points = monthly_Y_u_DJF_not_mean['phase_all'][key].shape[0]
#             AX.set_title(titles[A] , fontsize=35)

            lon_formatter = LongitudeFormatter(number_format='.1f',
                                               degree_symbol='',
                                               dateline_direction_label=True)
            lat_formatter = LatitudeFormatter(number_format='.1f',
                                              degree_symbol='')
            AX.xaxis.set_major_formatter(lon_formatter)
            AX.yaxis.set_major_formatter(lat_formatter)
            
            gl = AX.gridlines(crs=ccrs.PlateCarree(), draw_labels=True, dms=True, x_inline=False, y_inline=False)
            
            gl.top_labels = False
            gl.xlines = False
            gl.ylines = False
            gl.xlabel_style = {'size': 15, 'color': 'k'}
            gl.ylabel_style = {'size': 15, 'color': 'k'}
                       
        fig.suptitle('climatological zonal wind (contours) and jet location [rectangle box] \n [DJF, 1979-2019]', fontsize=20,  color='navy')    
#         cbar = fig.colorbar(im, ax=axs, location='bottom', shrink=0.7, aspect=35,)
#         cbar.ax.tick_params(labelsize=35)
        fig.show()


In [4]:
def plot_hist_map(jet_location = [15, 55, 40, 180], enso_key = 'neutral'):
    
    
        def jetval(field, jet_location):

            latt, xx, lonn = np.meshgrid(lat, range(field.shape[0]), lon)

            field_nan    = np.ma.masked_where( \
                                                ((latt < jet_location[0]) |  (latt > jet_location[1])) \
                                              | ((lonn < jet_location[2]) |  (lonn > jet_location[3])), field)

            jet_lat = np.ma.masked_where( field_nan == np.nanmax(field_nan), latt )
            jet_lon = np.ma.masked_where( field_nan == np.nanmax(field_nan), lonn )

            loc = np.nanmean(jet_lon, axis=(-1,-2)), np.nanmean(jet_lat, axis=(-1,-2))

            dicti = {'loc':loc, 'field':field_nan, \
                     'jet_max' :np.nanmax(field_nan, axis=(-1,-2)), \
                     'jet_mean':np.nanmean(field_nan, axis=(-1,-2)),\
                     'jet_box':jet_location}

            return dicti['jet_mean']


        def combine_bins(i1=0,i2=10, jet_location=jet_location):
            U = []
            for key in keys[i1:i2]:
                umean = jetval(monthly_Y_u_DJF_not_mean[enso_key]['phase_all'][key], jet_location = jet_location)
                U.append(umean)
            U = np.concatenate(U, axis=0)
            print (keys[i1], keys[i2])
            return U


        cmap2 = py.get_cmap("Reds_r")
        
        make_map_and_box(jet_location = jet_location, figsize=(10,4), xlims = ['0.0', '-2.5'], ylims = [-20, 90])

        keys = list(monthly_Y_u_DJF_not_mean[enso_key]['phase_all'].keys())

        py.figure(figsize=(14,6))

        umean = combine_bins(i1=12,i2=34,  jet_location=jet_location)
        p1=sns.distplot(umean, color='blue', kde=True, bins=50,\
                    hist_kws={"histtype": "bar", "linewidth": 1,
                                    "alpha": 1, "color": "blue", "edgecolor":"white"},\
                    kde_kws={"linewidth": 5, "color":"navy"}, label=r"MJO amp $\geq$ 1.1 [active mjo]")
        # py.axvline(stats.mode(umean)[1][0], color='blue')
        p1.legend(fontsize=20)

        py.tick_params(labelsize=20)

        umean = combine_bins(i1=0,i2=12, jet_location=jet_location)
        p2=sns.distplot(umean, color='cyan', kde=True, bins=50,\
                    hist_kws={"histtype": "bar", "linewidth": 1,
                              "alpha": 0.8, "color": "cyan", "edgecolor":"white"},\
                    kde_kws={"linewidth": 5, "color":"gray"}, label=r"MJO amp $\leq$ 1.1 [inactive mjo]")
        p2.legend(fontsize=20)
        # py.axvline(stats.mode(umean)[1][0], color='cyan')

        py.tick_params(labelsize=20)

        py.title('PDF of average jet strength in the Indo-Pacific region', fontsize=20)


        py.ylabel('pdf of number of days', fontsize=20)
        py.xlabel('Jet index [m/s]', fontsize=20)
        # py.legend(loc='best')

#         py.setp(p1._legend.get_texts(), fontsize=20)


In [8]:
# plot_hist_map(jet_location = [15, 54, 40, 180], enso_key = 'neutral')

In [9]:
# plot_hist_map(jet_location = [15, 54, 0, 359], enso_key = 'neutral')

In [10]:
%matplotlib inline
from ipywidgets import interactive

def f(la1, la2, lo1, lo2):
    plot_hist_map(jet_location = [la1, la2, lo1, lo2], enso_key = 'neutral')

interactive_plot = interactive(f, {'manual': True}, la1=(0, 80), la2=(0, 80), lo1=(0, 359), lo2=(0, 359), continuous_update=False)
# output = interactive_plot.children[-1]
# output.layout.height = '350px'
interactive_plot

interactive(children=(IntSlider(value=40, description='la1', max=80), IntSlider(value=40, description='la2', m…

In [18]:
%watermark -iv -m 

Compiler    : GCC 7.3.0
OS          : Linux
Release     : 4.18.0-372.9.1.el8.x86_64
Machine     : x86_64
Processor   : x86_64
CPU cores   : 64
Architecture: 64bit

scipy     : 1.6.2
hickle    : 3.4.5
seaborn   : 0.11.1
numpy     : 1.20.1
sys       : 3.8.8 (default, Apr 13 2021, 19:58:26) 
[GCC 7.3.0]
logging   : 0.5.1.2
netCDF4   : 1.5.6
matplotlib: 3.5.1
cartopy   : 0.18.0

